# Tutorial 4

In this tutorial we consider a generalised form of the elastic sea level problem and an associated reciprocity theorem. Doing so is necessary to obtain sensitivity kernels for other observables that do not depend only on the sea level change. 

Within the generalised sea level problem there are four force-terms:
- ```direct_load```: This is the same term as in the standard problem.
- ```displacement_load```: Physically, this corresponds to applying the traction of a surface load without considering the associated mass.
- ```gravitational_potential_load```: Physically this corresponds to the mass of a surface load without considering the associated traction. 
- ```angular_momentum_change```: This is a two-component vector expressing a jump in the systems angular momentum. 

Note that neither the displacement nor gravitational potential loads contribute to the mass balance between the oceans and ice sheets. Thus, changes to the mean sea level can only be generated by a non-zero direct load. 

Each of these loads can be provided to the ```__call__``` method of a ```FingerPrint``` instance, with the results being that due to the sum of all these generalised force terms. 

The reciprocity theorem for the generalised sea level equation states that
$$
\int_{\partial M} \zeta^{\dagger} \Delta SL \, \mathrm{d} S - \frac{1}{g}\int_{\partial M}(\zeta_{u}^{\dagger}\nabla\Phi \cdot \mathbf{u} + \zeta_{\phi}^{\dagger}\phi) \, \mathrm{d} S 
-\frac{1}{g} \Delta \mathbf{l}^{\dagger}\cdot \bm{\omega}
= \int_{\partial M} \zeta \Delta SL^{\dagger} \, \mathrm{d} S - \frac{1}{g}\int_{\partial M}(\zeta_{u}\nabla\Phi \cdot \mathbf{u}^{\dagger} + \zeta_{\phi}\phi^{\dagger}) \, \mathrm{d} S 
-\frac{1}{g} \Delta \mathbf{l}\cdot \bm{\omega}^{\dagger}.
$$
Here $\zeta$ is a direct load, $\zeta_{u}$ a displacement load, $\zeta_{\phi}$ a gravitational potential load, and $\Delta \mathbf{l}$ an angular momentum change, 
that result in the response $(\Delta SL, \mathbf{u}, \phi, \bm{\omega})$, and similarly for the daggered variables. 

### Checking the generalised reciprocity theorem

In the following example we generate two pairs a set of generalised forces and responses and check the reciprocity theorem holds to numerical precision. 


In [ ]:
# Import necessary modules for this notebook. 
import numpy as np
import matplotlib.pyplot as plt
from pyslfp import FingerPrint

# Set up the fingerprint instance. 
fingerprint = FingerPrint()
fingerprint.set_state_from_ice_ng()

In [19]:

# Function to generate random disk loads for convenience. 
def random_disk_load():
    radius = np.random.uniform(5,15)
    lat = np.random.uniform(-90,90)
    lon = np.random.uniform(-180,180)
    amplitude = np.random.uniform(-1,1)
    return fingerprint.disk_load(radius, lat, lon, amplitude)


# Set the first set of forces.
direct_load_1 = random_disk_load()
displacement_load_1 = random_disk_load()
gravitational_potential_load_1= random_disk_load()
angular_momentum_change_1 = np.random.randn(2)

# And now the corresponding response. 
(sea_level_change_1, 
 displacement_1, 
 gravity_potential_change_1, 
 angular_velocity_change_1) = fingerprint(direct_load=direct_load_1, 
                                          displacement_load=displacement_load_1, 
                                          gravitational_potential_load=gravitational_potential_load_1, 
                                          angular_momentum_change=angular_momentum_change_1) 


# Set the second set of forces.
direct_load_2 = random_disk_load()
displacement_load_2 = random_disk_load()
gravitational_potential_load_2= random_disk_load()
angular_momentum_change_2 = np.random.randn(2)

# And now the corresponding response. 
(sea_level_change_2, 
 displacement_2, 
 gravity_potential_change_2, 
 angular_velocity_change_2) = fingerprint(direct_load=direct_load_2, 
                                          displacement_load=displacement_load_2, 
                                          gravitational_potential_load=gravitational_potential_load_2, 
                                          angular_momentum_change=angular_momentum_change_2)                                           



# Get surface gravitational acceleration. 
g = fingerprint.gravitational_acceleration

lhs_integrand = (direct_load_2 * sea_level_change_1 
                -(1/g)*(g * displacement_load_2 * displacement_1 
                + gravitational_potential_load_2 * gravity_potential_change_1 ))
lhs = fingerprint.integrate(lhs_integrand) - np.dot(angular_momentum_change_2, angular_velocity_change_1)/g


rhs_integrand = (direct_load_1 * sea_level_change_2
                -(1/g)*(g * displacement_load_1 * displacement_2 
                + gravitational_potential_load_1 * gravity_potential_change_2 ))
rhs = fingerprint.integrate(rhs_integrand) - np.dot(angular_momentum_change_1, angular_velocity_change_2)/g


print(f'left hand side of identity {lhs}')
print(f'right hand side of identity {rhs}')
print(f'relative error in identity {np.abs(rhs-lhs)/np.abs(rhs)}')







left hand side of identity -16790522.676390108
right hand side of identity -16790529.411489233
relative error in identity 4.0112488180657887e-07
